## Latent Signal Detection

A "blackboard" style re-implementation of a drug-drug interaction discovery pipeline.

1. [Retrieve a reference set of drugs known to cause the **adverse event of interest**](#step1)
2. [Retrieve adverse event report data for these and a set of control drugs](#step2)
3. [Build a latent signal detection model on the reference set and predict DDIs](#step3)
4. [Corroborate putative ddis with clinical data](#step4) 
5. [Identify probable molecular mechanisms of action for the ddi](#step5)
6. _Retrieve related model systems (cell or animal models) that could be used to test the ddi predictions experimentally_


In [13]:
import csv
import gzip
import tqdm
import json
import heapq
import random
import urllib2
import requests

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import stats
from collections import defaultdict
from biothings_client import get_client

#from Authentication import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
from urllib2 import Request, urlopen, URLError, build_opener

from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3, RDF

%matplotlib inline

# Authentication Module
#get the tgt corresponding to the apikey
def get_tgt(apikey):
    tgt = requests.post("https://utslogin.nlm.nih.gov/cas/v1/api-key", data = {'apikey':apikey})
    TGT = tgt.content.split()[20]
    TGT = TGT[52:+len(TGT)-1]
    return TGT
#get a ticket corresponding to the tgt
def get_ticket(tgt):
    uri_login = "https://utslogin.nlm.nih.gov/cas/v1/tickets/" + tgt
    ticket = requests.post(uri_login, data = {'service':'http://umlsks.nlm.nih.gov'})
    return ticket.content

In [14]:
ADVERSE_EVENT_OF_INTEREST = 'qt prolonged'
#ADVERSE_EVENT_OF_INTEREST = 'Myocardial_Infarction'

# DEPRECATED
#ADVERSE_EVENT_OF_INTEREST = 'C0151878'

<a id='step1'></a>
**Step 1.** Given AE of interest build a set of reference drugs

ADVERSE_EVENT_OF_INTEREST may be a free text term like "QT prolonged" or one of the followed coded terms:
1. Myocardial_Infarction
2. Gastrointestinal_Bleeding
3. Kidney_Damage
4. Liver_Damage

In [15]:
# get mydrug client
md = get_client('drug')
mg = get_client('gene')

def names2rxnorm(drug_name_list):
    results = {}
    for _drug in drug_name_list:
        query_term = 'aeolus.drug_name:' + _drug
        _result = md.query(query_term, fields='aeolus.drug_code')
        if _result['hits'] != []:
            results[_drug] = [_record['aeolus']['drug_code'] for _record in _result['hits']]
        else:
            results[_drug] = []
    return results

def rxnorms2enzyme(rxnorm_list):
    results = {}
    for _rxnorm in rxnorm_list:
        query_term = 'aeolus.drug_code:' + _rxnorm
        _result = md.query(query_term, fields='drugbank.enzymes.name, drugbank.enzymes.uniprot')
        if _result['hits'] != []:
            results[_rxnorm] = [_record['drugbank'] for _record in _result['hits']]
        else:
            results[_rxnorm] = []
    return results

def rxnorms2enzymes(rxnorm_list):
    results = {}
    for _rxnorm in rxnorm_list:
        query_term = 'aeolus.drug_code:' + _rxnorm
        _result = md.query(query_term, fields='drugbank.enzymes.uniprot')
        if 'hits' in _result:
            results[_rxnorm] = [_record['uniprot'] for _record in _result['hits'][0]['drugbank']['enzymes']]
        else:
            results[_rxnorm] = ['no results found']
    return results

def compareenzymes(drug_name_list):
    results = rxnorms2enzymes(drug_name_list)
    uniprot_list = []
    for drug, uniprot in results.items():
        uniprot_list.append(uniprot)
    overlap = list(set(uniprot_list[0]) & set(uniprot_list[1]))
    if overlap:
        return {'overlap': overlap}
    else:
        print('no overlap')

def uniprot2symbol(uniprot_id_list):
    results = [['uniprot_id', 'gene_name', 'gene_symbol']]
    for _uniprot in uniprot_id_list:
        query_term = 'uniprot.Swiss-Prot:' + _uniprot
        _result = mg.query(query_term)
        if _result['hits'] != []:
            results.append([_uniprot, _result['hits'][0]['name'], _result['hits'][0]['symbol']])
    return results

#uniprot2symbol(compareenzymes(['36567', '38404'])['overlap'])

In [18]:
def name2rxnorm(drugname):
    annotations = annotator(drugname, ['RXNORM'])
    
    rxnorms = list()
    for annotation in annotations:
        rxnorms.append(annotations[0]['annotatedClass']['@id'].split('/')[-1])
    
    return rxnorms

def annotator(text, ontologies):
    REST_URL = "http://data.bioontology.org"
    API_KEY = 'b55cf758-dbd3-4959-a9c0-2e6bba5b5d8b'
    url = REST_URL + "/annotator?text=" + urllib2.quote(text) + "&ontologies=" + ','.join(ontologies)
    opener = urllib2.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    
    annotations = json.loads(opener.open(url).read())
    
    return annotations

def nsides_api(service, method, args = None):
    """
    Simple python API to access the NSIDES services and data.
    
    service   string, that can be one of three values: aeolus, sider, or omop
    method    string, for a list of methods see http://nsideseb-env.us-east-1.elasticbeanstalk.com
    args      dictionary, method parameters see http://nsideseb-env.us-east-1.elasticbeanstalk.com
    """
    NSIDES_API_URL = 'http://nsideseb-env.us-east-1.elasticbeanstalk.com/api/v1/'
    
    base_url = NSIDES_API_URL + 'query?service=%s&meta=%s' % (service, method)
    url = base_url
    if not args is None:
        url = base_url + '&' + '&'.join(['%s=%s' % (k,v) for k,v in args.items()])
    
    response = urllib2.urlopen(url).read()
    data = json.loads(response)
    
    return data

def deltaqt_api(drugs):
    """
    Simple python API to access the Delta QT services and data. 
    drugs     list of drug concept identifiers
    """
    DELTAQT_API_URL = 'http://deltaqt.org/api/v1/'
    url = DELTAQT_API_URL + 'query?drugs=%s' % ','.join(map(str,drugs))
    
    response = urllib2.urlopen(url).read()
    data = json.loads(response)
    
    return data

def AEtoCUI( aeName ):
    "Converts AE name to UMLS CUIs"
    cuis = []
    apikey = "6fd08e99-cb99-48be-937c-bbaa5205fd44"
    #AuthClient = Authentication(apikey)
    tgt = get_tgt(apikey)
    uri = "https://uts-ws.nlm.nih.gov"
    content_endpoint = "/rest/search/current?string="+aeName
    query = {'ticket':get_ticket(tgt)}
    r = requests.get(uri+content_endpoint,params=query)
    r.encoding = 'utf-8'
    items  = json.loads(r.text)
    jsonData = items["result"]
    for s in jsonData['results']:
        cuis.append(s['ui'])
    return cuis

In [19]:
if ADVERSE_EVENT_OF_INTEREST in ('Myocardial_Infarction', 'Gastrointestinal_Bleeding', 'Kidney_Damage', 'Liver_Damage'):
    # we use the OMOP reference standard
    data = nsides_api('omop', 'reference', {'q': ADVERSE_EVENT_OF_INTEREST})
    reference_byname = [(r['drugname'], int(r['causes_ae'])) for r in data['results']]
    
else:
    # we use SIDER
    reference_byname = list()
    for cui in AEtoCUI('qt prolonged'):
        data = nsides_api('sider', 'drugForEffect', {'q': cui})
        reference_byname.extend( [(r['drug_name'], 1) for r in data['results']] )
    
    if len(reference_byname) == 0:
        raise Exception("No available reference standard for cui: %s" % cui)
    

positives = list()
negatives = list()

drugnames  = [d for d,e in reference_byname]

# Uncomment the next line out to use biothings_client
# rxnorm_map = names2rxnorm(drugnames)

for drugname, causes_ae in tqdm(reference_byname):
    
    # Uncomment the next line to use NCBO Bioannotor to translate to rxnorm
    rxnorms = name2rxnorm(drugname)
    
    #print drugname, rxnorms
    # Comment the next line out to use biothings_client
    # rxnorms = rxnorm_map[drugname]
    
    for rxnorm in rxnorms:
        if causes_ae == 1:
            positives.append(int(rxnorm))
        else:
            negatives.append(int(rxnorm))

print "Produced reference with %d positive examples and %d negative examples." % (len(positives), len(negatives))

100%|██████████| 41/41 [00:26<00:00,  1.82it/s]


Produced reference with 40 positive examples and 0 negative examples.


In [20]:
# If there are no negative examples, we randomly pick 200 from what's available in AEOLUS
if len(negatives) == 0:
    data = nsides_api('aeolus', 'ingredientList')
    
    all_rxnorms = [r['concept_code'] for r in data['results']]
    available_negatives = (set(all_rxnorms) - set(positives))
    
    negatives = random.sample(available_negatives, 200)

print "Randomly selected %d negative examples from available drugs." % len(negatives)

Randomly selected 200 negative examples from available drugs.


<a id='step2'></a>
**Step 2.** Retrieve adverse event report data for these and a set of control drugs.

In [21]:
data = nsides_api('aeolus', 'ingredientList')

concept2name = dict()
all_drugs = list()
drug_mapping = dict()
concept2rxnorm = dict()

for r in data['results']:
    concept2name[r['concept_id']] = r['concept_name']
    all_drugs.append(r['concept_id'])
    drug_mapping[r['concept_code']] = r['concept_id']
    concept2rxnorm[r['concept_id']] = r['concept_code']

drug_mapping.items()[0]

(1114112, 40241205)

In [22]:
#query?service=aeolus&meta=reactionListMedDRA
data = nsides_api('aeolus', 'reactionListMedDRA')

all_outcomes = list()

for r in data['results']:
    all_outcomes.append(r['outcome_concept_id'])

len(all_outcomes)

7540

In [23]:
#query?service=aeolus&meta=drugpairReactionListMedDRA
data = nsides_api('aeolus', 'drugpairReactionListMedDRA')
all_pairoutcomes = list()
for r in data['results']:
    all_pairoutcomes.append(r['outcome_concept_id'])

len(all_pairoutcomes)

199

In [24]:
#query?service=aeolus&meta=drugpairList
data = nsides_api('aeolus', 'drugpairList')

all_drugpairs = list()

for r in data['results']:
    all_drugpairs.append((r['drug1_concept_id'], r['drug2_concept_id']))

len(all_drugpairs)

39800

In [25]:
#query?service=aeolus&meta=drugReactionCounts&q=0
data = nsides_api('aeolus', 'drugReactionCounts', {'q':0})

aeolus_drugReactionCounts = dict()
aeolus_drugReactionCounts['results'] = list()

for start in tqdm(range(0, data[0]['nrows'][0], 10000)):    
    aeolus_drugReactionCounts['results'].extend(data[0]['result'])    
    start = len(aeolus_drugReactionCounts['results'])
    data = nsides_api('aeolus', 'drugReactionCounts', {'q': start})

len(aeolus_drugReactionCounts['results'])

100%|██████████| 64/64 [05:59<00:00,  4.20s/it]


631743

In [26]:
# aeolus_drugReactionCounts['results'][0]

In [27]:
def make_lsd_training_table(pos_drugs = None, neg_drugs = None):

    pos_drugs_mapped = list()
    for pos_drug in pos_drugs:
        pos_drugs_mapped.append(drug_mapping[pos_drug])
        
    neg_drugs_mapped = list()
    for neg_drug in neg_drugs:
        neg_drugs_mapped.append(drug_mapping[neg_drug])
        
    outcomes = np.concatenate((np.ones(len(pos_drugs),np.bool), np.zeros(len(neg_drugs),np.bool)))
    
    training_drugs = pos_drugs_mapped + neg_drugs_mapped
    
    ae_frequencies_training = np.zeros((len(pos_drugs)+len(neg_drugs),len(all_pairoutcomes)))
    
    for r in aeolus_drugReactionCounts['results']:
        if not r['drug_concept_id'] in training_drugs:
            continue
        if not r['outcome_concept_id'] in all_pairoutcomes:
            continue
        
        
        i = training_drugs.index(r['drug_concept_id'])
        j = all_pairoutcomes.index(r['outcome_concept_id'])
        
        frequency = float(r['nreports'])/float(r['ndrugreports'])
        
        ae_frequencies_training[i,j] = frequency
                    
    return ae_frequencies_training, outcomes

In [28]:
pos_controls = [p for p in positives if p in drug_mapping]
neg_controls = [n for n in negatives if n in drug_mapping]

In [29]:
X,y = make_lsd_training_table(pos_controls, neg_controls)

print X.shape
print y.shape

(239L, 199L)
(239L,)


In [36]:
#query?service=aeolus&meta=drugpairReactionCounts&q=0
data = nsides_api('aeolus', 'drugpairReactionCounts', {'q':0})

aeolus_drugpairReactionCounts = dict()
aeolus_drugpairReactionCounts['results'] = list()

for start in tqdm(range(0, data[0]['nrows'][0], 10000)):    
    aeolus_drugpairReactionCounts['results'].extend(data[0]['results'])    
    start = len(aeolus_drugpairReactionCounts['results'])
    data = nsides_api('aeolus', 'drugpairReactionCounts', {'q': start})

len(aeolus_drugpairReactionCounts['results'])





  0%|          | 0/793 [00:00<?, ?it/s]



  0%|          | 1/793 [00:03<40:26,  3.06s/it]






  0%|          | 2/793 [00:06<40:20,  3.06s/it]



  0%|          | 3/793 [00:09<40:06,  3.05s/it]



  1%|          | 4/793 [00:12<40:12,  3.06s/it]



  1%|          | 5/793 [00:15<40:05,  3.05s/it]



  1%|          | 6/793 [00:18<40:23,  3.08s/it]



  1%|          | 7/793 [00:21<40:06,  3.06s/it]



  1%|          | 8/793 [00:24<40:09,  3.07s/it]



  1%|          | 9/793 [00:27<40:03,  3.07s/it]



  1%|▏         | 10/793 [00:30<40:11,  3.08s/it]



  1%|▏         | 11/793 [00:33<40:05,  3.08s/it]



  2%|▏         | 12/793 [00:36<39:27,  3.03s/it]



  2%|▏         | 13/793 [00:39<39:44,  3.06s/it]



  2%|▏         | 14/793 [00:42<39:40,  3.06s/it]



  2%|▏         | 15/793 [00:46<40:19,  3.11s/it]



  2%|▏         | 16/793 [00:49<40:33,  3.13s/it]



  2%|▏         | 17/793 [00:52<39:49,  3.08s/it]



  2%|▏         | 18/793 [00:55<39:32,  3.06s/it]



  2%|▏         | 19/793

 20%|█▉        | 156/793 [07:32<07:39,  1.38it/s]



 20%|█▉        | 157/793 [07:33<07:38,  1.39it/s]



 20%|█▉        | 158/793 [07:33<07:44,  1.37it/s]



 20%|██        | 159/793 [07:34<07:50,  1.35it/s]



 20%|██        | 160/793 [07:35<07:51,  1.34it/s]



 20%|██        | 161/793 [07:36<07:46,  1.36it/s]



 20%|██        | 162/793 [07:36<07:39,  1.37it/s]



 21%|██        | 163/793 [07:37<07:40,  1.37it/s]



 21%|██        | 164/793 [07:38<07:32,  1.39it/s]



 21%|██        | 165/793 [07:38<07:33,  1.38it/s]



 21%|██        | 166/793 [07:39<07:35,  1.38it/s]



 21%|██        | 167/793 [07:40<07:40,  1.36it/s]



 21%|██        | 168/793 [07:41<07:37,  1.37it/s]



 21%|██▏       | 169/793 [07:41<07:33,  1.37it/s]



 21%|██▏       | 170/793 [07:42<07:29,  1.39it/s]



 22%|██▏       | 171/793 [07:43<07:27,  1.39it/s]



 22%|██▏       | 172/793 [07:43<07:26,  1.39it/s]



 22%|██▏       | 173/793 [07:44<07:28,  1.38it/s]



 22%|██▏       | 174/793 [07:45<07:39,  1.35it

HTTPError: HTTP Error 404: Not Found

In [ ]:
aeolus_drugpairReactionCounts['results'][0]

In [ ]:
def make_lsd_testing_table():
    ae_frequencies_ddi_testing = np.zeros((len(all_drugpairs),len(all_pairoutcomes)))
    
    for r in tqdm(aeolus_drugpairReactionCounts['results']):
        
        drugpair = (r['drug1_concept_id'],r['drug2_concept_id'])
        if r['outcome_concept_id'] in all_pairoutcomes:
            x_index = all_drugpairs.index(drugpair)
            y_index = all_pairoutcomes.index(r['outcome_concept_id'])
            
            frequency = float(r['n_d1d2ae'])/float(r['n_d1d2'])
            
            ae_frequencies_ddi_testing[x_index,y_index] = frequency    
                
    return ae_frequencies_ddi_testing


In [ ]:
X_test = make_lsd_testing_table()

In [ ]:
print X.shape
print X_test.shape
print y.shape

In [ ]:
print len(positives)
print np.sum(y)

In [ ]:
X_test.sum(), X_test.mean()

<a id="step3"></a>
**Step 3.** Build a latent signal detection model on the reference set and predict DDIs

In [ ]:
def drugpair_predictions(X,y,X_test):
    rf = RandomForestClassifier(oob_score=True, random_state=12)
    rf.fit(X,y)
    predictions_proba = rf.predict_proba(X_test)[:,1]
    predictions = rf.predict(X_test)
    
    #print rf.score(X,y)
    oob_predictions = rf.oob_decision_function_[:,1]
    #print oob_predictions
    
    oob_predictions[ np.where( np.isnan(oob_predictions) == True)[0]] = 0
    
    #print oob_predictions
    
    print "OOB ROC:",roc_auc_score(y_true=y, y_score=oob_predictions)
    fprOOB, tprOOB, _ = roc_curve(y_true=y,y_score=oob_predictions)
    
    y_pred = np.zeros((len(predictions)))
    
    pos_drugs_mapped = list()
    for pos_drug in pos_controls:
        if pos_drug not in drug_mapping:
            continue
        
        pos_drugs_mapped.append(drug_mapping[pos_drug])
        
    for drugpair in all_drugpairs:         
        for drug in drugpair:
            if drug in pos_drugs_mapped:
                y_pred[all_drugpairs.index(drugpair)] = 1
                
    pred_sorted = predictions_proba
    
    
    top_ddi = sorted(zip(predictions_proba, all_drugpairs))[-50:]
    top_ddi.reverse()
    
    #print top_ddi
    
    for prob, ddi in top_ddi:
        idx = all_drugpairs.index(ddi)
        #print all_drugpairs[idx], predictions_proba[idx]
            
    print "Number of positive predictions:",np.sum(y_pred)
    
    print "Single-drug ROC:",roc_auc_score(y_true=y_pred,y_score=predictions_proba)
    
    fprSD, tprSD, _ = roc_curve(y_true=y_pred,y_score=predictions_proba)
    
    sns.set(style='ticks', font_scale=1.4)
    
    plt.figure(figsize=(6,6))
    plt.plot(fprSD,tprSD,label='One-of-pair (area= %0.2f)' % auc(fprSD,tprSD))
    plt.plot(fprOOB,tprOOB,label='OOB (area= %0.2f)' % auc(fprOOB,tprOOB))
    
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
    plt.legend(loc="lower right")
    
    sns.despine()
    
    return top_ddi

In [ ]:
top_ddi = drugpair_predictions(X,y,X_test)

<a id="step4"></a>
**Step 4.** Corroborate drug pair predictions using clinical data

Currently only works for QT prolongation data using the http://deltaQT.org resource.

In [ ]:
# Pull the top prediction
pair = top_ddi[10][1]

patient_counts = list()

for score, pair in top_ddi:
    
    drug1 = deltaqt_api([pair[0]])
    drug2 = deltaqt_api([pair[1]])
    pairdata = deltaqt_api(pair)
    
    patient_counts.append((len(pairdata['delta_qts']), len(drug1['delta_qts']), len(drug2['delta_qts']), pair))
    
    print "%20s %20s %10d %10d %10d" % (concept2name[pair[0]], concept2name[pair[1]], 
                                        len(pairdata['delta_qts']), 
                                        len(drug1['delta_qts']), 
                                        len(drug2['delta_qts']))

In [ ]:
pair = sorted(patient_counts)[-1][-1]

print pair

drug1 = deltaqt_api([pair[0]])
drug2 = deltaqt_api([pair[1]])
pairdata = deltaqt_api(pair)

In [ ]:
print "Found %d patients with available ECG data exposed to %s and %s" % \
    (len(pairdata['delta_qts']), concept2name[pair[0]], concept2name[pair[1]])

print "Found %d patients with available ECG data exposed to %s alone" % \
    (len(drug1['delta_qts']), concept2name[pair[0]])

print "Found %d patients with available ECG data exposed to %s alone" % \
    (len(drug2['delta_qts']), concept2name[pair[1]])


In [ ]:
sns.set(style='ticks',font_scale=1.4)

pair_exposed = [r['delta'] for r in pairdata['delta_qts']]
d1_exposed = [r['delta'] for r in drug1['delta_qts']]
d2_exposed = [r['delta'] for r in drug2['delta_qts']]

print "Combination vs %s: %s" % (concept2name[pair[0]], stats.mannwhitneyu(pair_exposed, d1_exposed))
print "Combination vs %s: %s" % (concept2name[pair[1]], stats.mannwhitneyu(pair_exposed, d2_exposed))
print "%s vs %s: %s" % (concept2name[pair[0]], concept2name[pair[1]], stats.mannwhitneyu(d1_exposed, d2_exposed))

plt.figure(figsize=(7,5))

plt.hist(d1_exposed, bins=30, lw=1, alpha=.6, edgecolor='white', normed=True)
plt.hist(d2_exposed, bins=30, lw=1, alpha=.6, edgecolor='white', normed=True)
plt.hist(pair_exposed, bins=30, lw=1, alpha=.6, edgecolor='white', normed=True)

plt.title('Response to %s + %s' % (concept2name[pair[0]], concept2name[pair[1]]))
plt.xlabel('Change in QT')
plt.ylabel('Frequency')

plt.legend(['Combination', concept2name[pair[0]], concept2name[pair[1]]])

sns.despine()

<a id="step5"></a>
**Step 5.** Identify probable molecular mechanisms of action for the ddi

In [ ]:
pair[0], concept2rxnorm[pair[0]], pair[1], concept2rxnorm[pair[1]]

In [ ]:
rxnorms2enzyme([str(concept2rxnorm[pair[0]])])

In [ ]:
rxnorms2enzyme([str(concept2rxnorm[pair[1]])])

In [ ]:
sparql = SPARQLWrapper("http://bio2rdf.org/sparql")
sparql.setQuery("""
PREFIX dct: <http://purl.org/dc/terms/>
SELECT distinct str(?phenotype_label) as ?pl
WHERE {
    ?drug <http://bio2rdf.org/drugbank_vocabulary:target> ?target ;
        <http://bio2rdf.org/drugbank_vocabulary:x-atc> ?atc.
    FILTER(?atc = <http://bio2rdf.org/atc:L01XE01>)

    ?target <http://bio2rdf.org/drugbank_vocabulary:x-hgnc> ?hgnc .

    ?hgnc <http://bio2rdf.org/hgnc_vocabulary:x-mgi> ?marker .

    ?model <http://bio2rdf.org/mgi_vocabulary:marker> ?marker .
    ?model <http://bio2rdf.org/mgi_vocabulary:allele> ?all .
    ?all <http://bio2rdf.org/mgi_vocabulary:allele-attribute> ?allele_type .
    ?model <http://bio2rdf.org/mgi_vocabulary:phenotype> ?phenotypes .
    FILTER (str(?allele_type) = "Null/knockout")
    
    ?p owl:sameAs ?phenotypes .
    ?p rdfs:label ?phenotype_label .
}
""")

print '*** JSON Example'
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print result["pl"]["value"]

print '\n*** N3 Example'
sparql.setReturnFormat(N3)
results = sparql.query().convert()
print results

print '*** XML Example'
sparql.setReturnFormat(XML)
results = sparql.query().convert()
print results.toxml()